Visualization example of a very simple two-dimensional classification problem.

The negative (-1) class is sampled within a circle with radius 1 while the positive (+1) class is sampled from a ring with inner radius 0.95 and outer radius sqrt(2)

In [ ]:
import numpy as np

from explainpolysvm import expsvm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


In [ ]:
save_figs = False

In [ ]:
n_train_per_class = 8

# Radii for rings
r_min1 = 1.
r_max1 = 1.
r_min2 = 3
r_max2 = 3
center = np.array([3,2])

# Sample from classes
phi_train1 = 2 * np.pi * np.arange(0, 1, 1/n_train_per_class)#np.random.rand(n_train_per_class)
r_train1 = r_min1 + (r_max1 - r_min1) * np.random.rand(n_train_per_class, 1)
X_train1 = center +np.multiply(r_train1, np.transpose(np.array((np.cos(phi_train1), np.sin(phi_train1)))))

phi_train2 = 2 * np.pi * np.arange(0, 1, 1/n_train_per_class)#np.random.rand(3*n_train_per_class)
r_train2 = r_min2 + (r_max2 - r_min2) * np.random.rand(n_train_per_class, 1)
X_train2 = center + np.multiply(r_train2, np.transpose(np.array((np.cos(phi_train2), np.sin(phi_train2)))))

# # X_train1 = np.array([[0,0]])
# X_train1 = np.array([[0.5,0], [-0.5, 0]])#, [0, 0.5], [0, -0.5]])
# X_train2 = np.array([[1.5,0], [-1.5, 0]])#, [0, 1.5], [0, -1.5]])

# # rotation
# theta = np.pi/4
# rot_matrix = np.array([[np.cos(theta), -np.sin(theta)],
#                        [np.sin(theta), np.cos(theta)]])

# # Rotate
# X_train1 = np.matmul(rot_matrix, X_train1.T).T
# X_train2 = np.matmul(rot_matrix, X_train2.T).T

X = np.concatenate((X_train1, X_train2), axis=0)
# y = np.concatenate((np.ones(n_train_per_class), -np.ones(n_train_per_class)))
y = np.concatenate((np.ones(len(X_train1)), -np.ones(len(X_train2))))
# y = [-1, -1, -1, 1, 1, 1]

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.scatter(x=X_train1[:,0], y=X_train1[:,1], color=[0.5, 0.5, 0.5])
ax.scatter(x=X_train2[:,0], y=X_train2[:,1], color=[0.2, 0.2, 0.2], marker='x')
ax.set_aspect('equal', adjustable='box')
plt.legend(['class 1', 'class -1'])
if save_figs:
    plt.savefig('./images/training_data_2d.png', dpi=200,facecolor='white')
plt.draw()

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=50, random_state=10)
X_train, y_train = X, y
# Fit SVM
C = 10
degree = 2
gamma = 'scale'
r = 1*np.sqrt(2)

# Fit SVM
kernel = 'poly'
model = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma, coef0=r)
model.fit(X_train, y_train)

sv = model.support_vectors_
dual_coef = np.squeeze(model.dual_coef_)
intercept = model.intercept_[0]
kernel_gamma = model._gamma

es = expsvm.ExPSVM(sv=sv, dual_coef=dual_coef, intercept=intercept,
                kernel_d=degree, kernel_r=r, kernel_gamma=kernel_gamma)
es.transform_svm()

Test performance

In [ ]:
# y_pred = np.sign(es.decision_function(x=X_test))
# acc = np.sum(y_pred==y_test)/y_test.size
# print('Test accuracy: {}'.format(acc))

Feature importance

In [ ]:
_ = es.plot_model_bar(n_features=20, magnitude=False, show=False, figsize=(4,3))
if save_figs:
    plt.savefig('./images/feature_importance_2d.png', dpi=200, facecolor='white')
plt.show()

In [ ]:
f = es.feature_importance(format_names=False, include_intercept=True, magnitude=False)

def calc_boundary_for_2d_sv_with_2_features(x, is_x0, feat_imp, decimals: int = None):
    """Calcuate the roots of a two-dimensional quadratic (or linear) function.
    """
    a = feat_imp[0][np.where(feat_imp[1] == 'intercept')[0][0]]
    if is_x0:
        b = feat_imp[0][np.where(feat_imp[1] == '0')[0][0]]
        c = feat_imp[0][np.where(feat_imp[1] == '1')[0][0]]
        d = feat_imp[0][np.where(feat_imp[1] == '1,1')[0][0]]
        e = feat_imp[0][np.where(feat_imp[1] == '0,0')[0][0]]
    else:
        c = feat_imp[0][np.where(feat_imp[1] == '0')[0][0]]
        b = feat_imp[0][np.where(feat_imp[1] == '1')[0][0]]
        e = feat_imp[0][np.where(feat_imp[1] == '1,1')[0][0]]
        d = feat_imp[0][np.where(feat_imp[1] == '0,0')[0][0]]
    f = feat_imp[0][np.where(feat_imp[1] == '0,1')[0][0]]

    if decimals is not None:
        a = np.round(a, decimals)
        b = np.round(b, decimals)
        c = np.round(c, decimals)
        d = np.round(d, decimals)
        e = np.round(e, decimals)
        f = np.round(f, decimals)

    sqrt_content = (c + f*x)**2 - 4*d*(a + x*(b + e*x))
    # print(sqrt_content)
    x1_p, x1_m = np.nan, np.nan
    if d != 0:
        if sqrt_content >=-np.inf:
            x1_p = -(c + f*x + np.sqrt(sqrt_content))/(2*d)
            x1_m = -(c + f*x - np.sqrt(sqrt_content))/(2*d)
    else:
        if c + f*x != 0:
            x1_p = - (a + x*(b + e*x))/(c + f*x)
            x1_m = np.nan
            
    return [x1_p, x1_m]


In [ ]:
n_step = 201
x0_min = -1
x0_max = 9
x1_min = -2
x1_max = 8
x0_space = np.linspace(x0_min, x0_max, n_step)
x1_space = np.linspace(x1_min, x1_max, n_step)
x0_grid, x1_grid = np.meshgrid(x0_space, x1_space)
# x_grid = x_grid.flatten()\\\
# x1_grid = x1_grid.flatten()\\\
x0x1_grid = np.concatenate((x0_grid.flatten()[:, np.newaxis], x1_grid.flatten()[:, np.newaxis]), axis=1)
y_pred = es.decision_function(x0x1_grid)

x1_boundary = np.array([calc_boundary_for_2d_sv_with_2_features(x0_b, True, f, 3) for x0_b in x0_space])
x0_boundary = np.array([calc_boundary_for_2d_sv_with_2_features(x1_b, False, f, 3) for x1_b in x1_space])


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,5))
delta = (np.max(y_pred) - np.min(y_pred))/20
cmap = ax.contourf(x0_grid, x1_grid, y_pred.reshape((n_step, n_step)), 
                   levels=np.arange(np.min(y_pred), np.max(y_pred)+delta, delta))

ax.scatter(x=X_train1[:,0], y=X_train1[:,1], color='k', marker='o')
ax.scatter(x=X_train2[:,0], y=X_train2[:,1], color='k', marker='x')

plt.plot(x0_space, x1_boundary[:, 0], 'k')
plt.plot(x0_space, x1_boundary[:, 1], 'k')
plt.plot(x0_boundary[:, 0], x1_space, 'k')
plt.plot(x0_boundary[:, 1], x1_space, 'k')

ax.set_aspect('equal', adjustable='box')
plt.legend(['class 1', 'class -1', 'decision boundary'])
plt.colorbar(cmap, ax=ax)
plt.xlim([x0_min, x0_max])
plt.ylim([x1_min, x1_max])
plt.show()

In [ ]:
es.feature_importance(magnitude=False)

In [ ]:
model_y_pred = model.decision_function(x0x1_grid)
print(model_y_pred.max(), model.intercept_)

In [ ]:

def calc_x0(x1):
    return 3 + np.sqrt(5 - (x1-2)**2), 3 - np.sqrt(5 - (x1-2)**2)


fig, ax = plt.subplots(1,1, figsize=(5,5))
cmap = plt.contourf(x0_grid, x1_grid, 
                    (y_pred>0).reshape((n_step, n_step)).astype(float)
                    -(y_pred<0).reshape((n_step, n_step)).astype(float), cmap='binary', levels=1)
plt.colorbar(cmap)
ax.scatter(x=X_train1[:,0], y=X_train1[:,1], color='k', marker='o')
ax.scatter(x=X_train2[:,0], y=X_train2[:,1], color='k', marker='x')
plt.plot(x0_space, x1_boundary[:, 0], 'k')
plt.plot(x0_space, x1_boundary[:, 1], 'k')
plt.plot(x0_boundary[:, 0], x1_space, 'k')
plt.plot(x0_boundary[:, 1], x1_space, 'k')
ax.set_aspect('equal', adjustable='box')
plt.legend(['class 1', 'class -1', 'decision boundary'])
for x1 in [0, 1, 2, 3, 4]:
    print(calc_x0(x1), x1)
    plt.scatter(calc_x0(x1)[0], x1, marker='d', color='k')
    plt.scatter(calc_x0(x1)[1], x1, marker='d', color='k')
# plt.xlim([x0_min, x0_max])
# plt.ylim([x1_min, x1_max])
plt.clim([-1.5, 1.5])
# plt.grid()
plt.show()

The two concentric circles are centered on (3,2) and the decision boundary is $b(x0, x1) = -2 + 1.5*x_0 + 1*x_1 - 0.25*x_0^2 - 0.25*x_1^2  = 0$, or, equivalently $0 = -8 + x_0(6 - x_0) + x_1(4 - x_1)$ and
$(x_0 - 3)^2 + (x_1 - 2)^2 = 5$.


Maximum decision function value is reached at (3,2), we reach zero, for example at (1,2)